In [25]:
class Game:
    """A game is similar to a problem, but it has a utility for each
    state and a terminal test instead of a path cost and a goal
    test. To create a game, subclass this class and implement actions,
    result, utility, and terminal_test. You may override display and
    successors or you can inherit their default methods. You will also
    need to set the .initial attribute to the initial state; this can
    be done in the constructor."""

    def actions(self, state):
        """Return a list of the allowable moves at this point."""
        raise NotImplementedError

    def result(self, state, move):
        """Return the state that results from making a move from a state."""
        raise NotImplementedError

    def utility(self, state, player):
        """Return the value of this final state to player."""
        raise NotImplementedError

    def terminal_test(self, state):
        """Return True if this is a final state for the game."""
        return not self.actions(state)

    def to_move(self, state):
        """Return the player whose move it is in this state."""
        return state.to_move

    def display(self, state):
        """Print or otherwise display the state."""
        print(state)

    def __repr__(self):
        return '<{}>'.format(self.__class__.__name__)

    def play_game(self, *players):
        """Play an n-person, move-alternating game."""
        state = self.initial
        while True:
            for player in players:
                move = player(self, state)
                state = self.result(state, move)
                if self.terminal_test(state):
                    self.display(state)
                    return self.utility(state, self.to_move(self.initial))







In [26]:
import copy
import itertools
import random
from collections import namedtuple
import numpy as np



def minmax_decision(state, game):
    """Given a state in a game, calculate the best move by searching
    forward all the way to the terminal states. [Figure 5.3]"""

    player = game.to_move(state)

    def max_value(state):
        if game.terminal_test(state):
            return game.utility(state, player)
        v = -np.inf
        for a in game.actions(state):
            v = max(v, min_value(game.result(state, a)))
        return v

    def min_value(state):
        if game.terminal_test(state):
            return game.utility(state, player)
        v = np.inf
        for a in game.actions(state):
            v = min(v, max_value(game.result(state, a)))
        return v

    # Body of minmax_decision:
    return max(game.actions(state), key=lambda a: min_value(game.result(state, a)))





In [27]:
class Fig52Game(Game):
    """The game represented in [Figure 5.2]. Serves as a simple test case."""

    succs = dict(A=dict(a1='B', a2='C', a3='D'),
                 B=dict(b1='B1', b2='B2', b3='B3'),
                 C=dict(c1='C1', c2='C2', c3='C3'),
                 D=dict(d1='D1', d2='D2', d3='D3'))
    utils = dict(B1=3, B2=12, B3=8, C1=2, C2=4, C3=6, D1=14, D2=5, D3=2)
    initial = 'A'

    def actions(self, state):
        return list(self.succs.get(state, {}).keys())

    def result(self, state, move):
        return self.succs[state][move]

    def utility(self, state, player):
        if player == 'MAX':
            return self.utils[state]
        else:
            return -self.utils[state]

    def terminal_test(self, state):
        return state not in ('A', 'B', 'C', 'D')

    def to_move(self, state):
        return 'MIN' if state in 'BCD' else 'MAX'


class Fig52Extended(Game):
    """Similar to Fig52Game but bigger. Useful for visualisation"""

    succs = dict(A=dict(a1='B', a2='C', a3='D'),
                 B=dict(b1='B1', b2='B2', b3='B3'),
                 C=dict(c1='C1', c2='C2', c3='C3', c4='E'),
                 D=dict(d1='D1', d2='D2', d3='D3'),
                 E=dict(e1='E1', e2='E2', e3='E3', e4='F'),
                 F=dict(f1='F1', f2='F2', f3='F3'))
    utils = dict(B1=3, B2=12, B3=8, C1=2, C2=4, C3=6, D1=14, D2=5, D3=2, E1=7, E2=8, E3=9, F1=4, F2=3, F3=11)

    def actions(self, state):
        return sorted(list(self.succs.get(state, {}).keys()))

    def result(self, state, move):
        return self.succs[state][move]

    def utility(self, state, player):
        if player == 'MAX':
            return self.utils[state]
        else:
            return -self.utils[state]

    def terminal_test(self, state):
        return state not in ('A', 'B', 'C', 'D', 'E', 'F')

    def to_move(self, state):
        return 'MIN' if state in 'BCDF' else 'MAX'

In [28]:
game_extended = Fig52Extended()

In [29]:
minmax_decision('E', game_extended)

'e3'

In [30]:
game = Fig52Game()

In [31]:
minmax_decision('C', game)

'c1'

In [32]:
A=dict(a1='B', a2='C', a3='D')

In [33]:
dict(A=dict(a1='B', a2='C', a3='D'))

{'A': {'a1': 'B', 'a2': 'C', 'a3': 'D'}}

In [34]:
{i: dict(l=i * 3 + 1, m=i * 3 + 2, r=i * 3 + 3) for i in range(13)}

{0: {'l': 1, 'm': 2, 'r': 3},
 1: {'l': 4, 'm': 5, 'r': 6},
 2: {'l': 7, 'm': 8, 'r': 9},
 3: {'l': 10, 'm': 11, 'r': 12},
 4: {'l': 13, 'm': 14, 'r': 15},
 5: {'l': 16, 'm': 17, 'r': 18},
 6: {'l': 19, 'm': 20, 'r': 21},
 7: {'l': 22, 'm': 23, 'r': 24},
 8: {'l': 25, 'm': 26, 'r': 27},
 9: {'l': 28, 'm': 29, 'r': 30},
 10: {'l': 31, 'm': 32, 'r': 33},
 11: {'l': 34, 'm': 35, 'r': 36},
 12: {'l': 37, 'm': 38, 'r': 39}}

In [35]:
def alpha_beta_search(state, game):
    """Search game to determine best action; use alpha-beta pruning.
    As in [Figure 5.7], this version searches all the way to the leaves."""

    player = game.to_move(state)

    # Functions used by alpha_beta
    def max_value(state, alpha, beta):
        if game.terminal_test(state):
            return game.utility(state, player)
        v = -np.inf
        for a in game.actions(state):
            v = max(v, min_value(game.result(state, a), alpha, beta))
            if v >= beta:
                return v
            alpha = max(alpha, v)
        return v

    def min_value(state, alpha, beta):
        if game.terminal_test(state):
            return game.utility(state, player)
        v = np.inf
        for a in game.actions(state):
            v = min(v, max_value(game.result(state, a), alpha, beta))
            if v <= alpha:
                return v
            beta = min(beta, v)
        return v

    # Body of alpha_beta_search:
    best_score = -np.inf
    beta = np.inf
    best_action = None
    for a in game.actions(state):
        v = min_value(game.result(state, a), best_score, beta)
        if v > best_score:
            best_score = v
            best_action = a
    return best_action

In [36]:
alpha_beta_search('B', game_extended)

'b1'

In [37]:
def WalkSAT(clauses, p=0.5, max_flips=10000):
    """Checks for satisfiability of all clauses by randomly flipping values of variables
    """
    # Set of all symbols in all clauses
    symbols = {sym for clause in clauses for sym in prop_symbols(clause)}
    # model is a random assignment of true/false to the symbols in clauses
    model = {s: random.choice([True, False]) for s in symbols}
    for i in range(max_flips):
        satisfied, unsatisfied = [], []
        for clause in clauses:
            (satisfied if pl_true(clause, model) else unsatisfied).append(clause)
        if not unsatisfied:  # if model satisfies all the clauses
            return model
        clause = random.choice(unsatisfied)
        if probability(p):
            sym = random.choice(list(prop_symbols(clause)))
        else:
            # Flip the symbol in clause that maximizes number of sat. clauses
            def sat_count(sym):
                # Return the the number of clauses satisfied after flipping the symbol.
                model[sym] = not model[sym]
                count = len([clause for clause in clauses if pl_true(clause, model)])
                model[sym] = not model[sym]
                return count
            sym = argmax(prop_symbols(clause), key=sat_count)
        model[sym] = not model[sym]
    # If no solution is found within the flip limit, we return failure
    return None

In [38]:
def WalkSAT_CNF(sentence, p=0.5, max_flips=10000):
    return WalkSAT(conjuncts(to_cnf(sentence)), 0, max_flips)

In [39]:
!git clone https://github.com/aimacode/aima-python.git

Cloning into 'aima-python'...
remote: Enumerating objects: 5095, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 5095 (delta 0), reused 1 (delta 0), pack-reused 5092
Receiving objects: 100% (5095/5095), 17.71 MiB | 14.42 MiB/s, done.
Resolving deltas: 100% (3416/3416), done.


In [40]:
cd aima-python/

/content/aima-python


In [41]:
from utils import *

In [42]:
!pip install ipythonblocks

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.6 MB 4.5 MB/s 


In [43]:
import heapq
import itertools
import random
from collections import defaultdict, Counter
from logic import to_cnf, prop_symbols
from utils import remove_all, unique, first, probability, isnumber, issequence, Expr, expr, subexpressions, extend

In [44]:
def WalkSAT(clauses, p=0.5, max_flips=10000):
    """Checks for satisfiability of all clauses by randomly flipping values of variables
    >>> WalkSAT([A & ~A], 0.5, 100) is None
    True
    """
    # Set of all symbols in all clauses
    symbols = {sym for clause in clauses for sym in prop_symbols(clause)}
    # model is a random assignment of true/false to the symbols in clauses
    model = {s: random.choice([True, False]) for s in symbols}
    for i in range(max_flips):
        satisfied, unsatisfied = [], []
        for clause in clauses:
            (satisfied if pl_true(clause, model) else unsatisfied).append(clause)
        if not unsatisfied:  # if model satisfies all the clauses
            return model
        clause = random.choice(unsatisfied)
        if probability(p):
            sym = random.choice(list(prop_symbols(clause)))
        else:
            # Flip the symbol in clause that maximizes number of sat. clauses
            def sat_count(sym):
                # Return the the number of clauses satisfied after flipping the symbol.
                model[sym] = not model[sym]
                count = len([clause for clause in clauses if pl_true(clause, model)])
                model[sym] = not model[sym]
                return count

            sym = max(prop_symbols(clause), key=sat_count)
        model[sym] = not model[sym]
    # If no solution is found within the flip limit, we return failure
    return None

In [45]:
X1, X2, X3, X4, X5, X6 = expr('X1, X2, X3, X4, X5, X6')

In [46]:
A

{'a1': 'B', 'a2': 'C', 'a3': 'D'}

In [47]:
from logic import pl_true, dissociate
from numpy import argmax
from logic import *

WalkSAT([A, B, ~C, D], 0.5, 100)


{C: False, B: True, A: True, D: True}

In [48]:
def WalkSAT_CNF(sentence, p=0.5, max_flips=10000):
    return WalkSAT(conjuncts(to_cnf(sentence)), 0, max_flips)

In [49]:
sentence_1 = A |'<=>'| B
sentence_2 = (A & B) | (C & ~A) | (B & ~D)
sentence_3 = (A | (B & C)) |'<=>'| ((A | B) & (A | C))

In [50]:
sentence_1


(A <=> B)

In [51]:
WalkSAT_CNF(sentence_1)
WalkSAT_CNF(sentence_2)
WalkSAT_CNF(sentence_3)


{C: True, B: True, A: False}



```
Here are the 3-CNF inputs used

1) (x1 v x2 v x3) ^ (~x1 v ~x2) ^ (~x1 v ~x3) ^ (~x2 v x3) ^ (x1 v ~x3)

2) (~x1 v x2) ^ (~x2 v ~x3 v x4) ^ (~x3 v ~x4 v x5) ^ (x1 v ~x4 v x5) ^ ~(x1 v x2 v x3)

3) (x1 ^ ~x2 ^ ~x3) v (~x1 ^ x2 ^ x3) v (x1 ^ ~x2 ^ x3) v (x1 ^ x2 ^ ~x3)

4) (~x1 v x2 v x3) ^ (~x1 v ~x2 v x3) ^ (~x1 v ~x2 v ~x3) ^ (x2 v ~x3) ^ (x3 v x4)

5) (~x1 v x2) ^ (~x1 v ~x2 v x3) ^ (~x2 v ~x3 v x4) ^ (~x3 v ~x4 v x5) ^ (~x4 v ~x5 v x6)

6) (x1 ^ ~x2 ^ x3) v (~x1 ^ x2 ^ ~x3) v (x1 ^ x2 ^ x3) v (~x1 ^ ~x2 ^ ~x3) v (x2 v x3)

# 7) (~x1 v ~x2 v x3) ^ (~x1 v x2 v ~x3) ^ (x1 v ~x2 v ~x3) ^ (~x3 v x4) ^ (x1 v x2 v x3)

8) (~x1 v ~x2) ^ (~x1 v ~x2 v x3) ^ (~x1 v ~x2 v ~x3) ^ (~x2 v ~x3 v ~x4) ^ (x4 v ~x5)

9) (x1 ^ x2 ^ ~x3) v (~x1 ^ x2 ^ x3) v (~x1 ^ ~x2 ^ x3) v (~x1 ^ ~x2 ^ ~x3) v (x2 v ~x3)

10) (~x1 v ~x2 v ~x3) ^ (x1 v ~x2 v x3) ^ (~x1 v x2 v x3) ^ (~x2 v ~x3 v x4) ^ (x1 v ~x2 v ~x3)
```



In [52]:
sent1 = ((X1 | X2 | X3) & (~X1 | ~X2) ^ (~X1 | ~X3) & (~X2 | X3) & (X1 | ~X3))

In [53]:
WalkSAT_CNF(sent1)

{X2: True, X1: False, X3: True}

In [54]:
sent2 = ((~X1 | X2) & (~X2 | ~X3 | X4) & (~X3 | ~X4 | X5) & (X1 | ~X4 | X5) & ~(X1 | X2 | X3))

In [55]:
sent2

(((((~X1 | X2) & ((~X2 | ~X3) | X4)) & ((~X3 | ~X4) | X5)) & ((X1 | ~X4) | X5)) & ~((X1 | X2) | X3))

In [56]:
WalkSAT_CNF(sent2)

{X4: False, X1: False, X5: True, X2: False, X3: False}

In [57]:
# 3) (x1 ^ ~x2 ^ ~x3) v (~x1 ^ x2 ^ x3) v (x1 ^ ~x2 ^ x3) v (x1 ^ x2 ^ ~x3)
sent3 = ((X1 & ~X2 & ~X3) | (~X1 & X2 & X3) | (X1 & ~X2 & X3) | (X1 & X2 & ~X3))

In [58]:
WalkSAT_CNF(sent3)

{X2: True, X3: True, X1: False}

In [59]:
# (~x1 v x2 v x3) ^ (~x1 v ~x2 v x3) ^ (~x1 v ~x2 v ~x3) ^ (x2 v ~x3) ^ (x3 v x4)

sent4 = ((~X1 | X2 | X3) & (~X1 | ~X2 | X3) & (~X1 | ~X2 | ~X3) & (X2 | ~X3) & (X3 | X4))

In [60]:
sent4

((((((~X1 | X2) | X3) & ((~X1 | ~X2) | X3)) & ((~X1 | ~X2) | ~X3)) & (X2 | ~X3)) & (X3 | X4))

In [61]:
WalkSAT_CNF(sent4)

{X2: True, X4: False, X1: False, X3: True}

In [62]:
# (~x1 v x2) ^ (~x1 v ~x2 v x3) ^ (~x2 v ~x3 v x4) ^ (~x3 v ~x4 v x5) ^ (~x4 v ~x5 v x6)

sent5 = ((~X1 | X2) & (~X1 | ~X2 | X3) & (~X2 | ~X3 |X4) & (~X3 | ~X4 | X5) & (~X4 | ~X5 | X6))

In [63]:
sent5

(((((~X1 | X2) & ((~X1 | ~X2) | X3)) & ((~X2 | ~X3) | X4)) & ((~X3 | ~X4) | X5)) & ((~X4 | ~X5) | X6))

In [64]:
WalkSAT_CNF(sent5)

{X4: False, X1: False, X6: False, X5: False, X2: False, X3: True}

In [65]:
# (x1 ^ ~x2 ^ x3) v (~x1 ^ x2 ^ ~x3) v (x1 ^ x2 ^ x3) v (~x1 ^ ~x2 ^ ~x3) v (x2 v x3)

sent6 = ((X1 & ~X2 & X3) | (~X1 & X2 & ~X3) | (X1 & X2 & X3) | (~X1 & ~X2 & ~X3) | (X2 | X3))

In [66]:
WalkSAT_CNF(sent6)

{X2: False, X1: False, X3: True}

In [67]:
# (~x1 v ~x2 v x3) ^ (~x1 v x2 v ~x3) ^ (x1 v ~x2 v ~x3) ^ (~x3 v x4) ^ (x1 v x2 v x3)
sent7 = ((~X1 | ~X2 | X3) & (~X1 | X2 | ~X3) & (X1 | ~X2 | ~X3) & (~X3 | X4) & (X1 | X2 |X3))

In [68]:
WalkSAT_CNF(sent7)

{X2: False, X4: True, X1: True, X3: False}

In [69]:
# (~x1 v ~x2) ^ (~x1 v ~x2 v x3) ^ (~x1 v ~x2 v ~x3) ^ (~x2 v ~x3 v ~x4) ^ (x4 v ~x5)

sent8 = ((~X1 | ~X2) & (~X1 | ~X2 | X3) & (~X1 | ~X2 | ~X3) & (~X2 | ~X3 | ~X4) & (X4 | ~X5))

In [70]:
WalkSAT_CNF(sent8)

{X4: False, X1: False, X5: False, X2: True, X3: True}

In [71]:
# (x1 ^ x2 ^ ~x3) v (~x1 ^ x2 ^ x3) v (~x1 ^ ~x2 ^ x3) v (~x1 ^ ~x2 ^ ~x3) v (x2 v ~x3)
sent9 = ((X1 & X2 & ~X3) | (~X1 & X2 & X3) | (~X1 & ~X2 & X3) | (~X1 & ~X2 & ~X3) | (X2 | ~X3))

In [72]:
WalkSAT_CNF(sent9)

{X2: False, X1: True, X3: False}

In [73]:
# (~x1 v ~x2 v ~x3) ^ (x1 v ~x2 v x3) ^ (~x1 v x2 v x3) ^ (~x2 v ~x3 v x4) ^ (x1 v ~x2 v ~x3)

sent10 = ((~X1 | ~X2 | ~X3) & (X1 | ~X2 | X3) & (~X1 | X2 | X3) & (~X2 | ~X3 | X4) & (X1 | ~X2 | ~X3))

In [74]:
WalkSAT_CNF(sent10)

{X2: False, X4: True, X1: False, X3: False}

In [75]:
%%timeit

WalkSAT_CNF(sent1)
WalkSAT_CNF(sent2)
WalkSAT_CNF(sent3)
WalkSAT_CNF(sent4)
WalkSAT_CNF(sent5)
WalkSAT_CNF(sent6)
WalkSAT_CNF(sent7)
WalkSAT_CNF(sent8)
WalkSAT_CNF(sent9)
WalkSAT_CNF(sent10)

24.9 ms ± 9.06 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
